In [ ]:
# Imports

import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
from glob import glob
from matplotlib import pyplot as plt
from datetime import datetime

from __future__ import print_function
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import model_from_json
import os

In [ ]:
# Directory Path:
IMG_PATH = "../data/pets/train/"
LABEL_PATH = "../data/pets/train_label.txt"
TRAIN_VAL_SPLIT = 0.8

# Constants
NUM_IMG = 2000

In [ ]:
# Image Standardization
def get_resized_images(path):
    imlist = []
    for i in range(NUM_IMG):
        path_str = path + "pet" + str(i+1) + ".jpg"
        im = cv2.imread(path_str)
        resized_im = cv2.resize(im, (128, 128))
        imlist.append(resized_im)  
    return imlist

# Read In Labels
def get_labels(path):
    labels = np.loadtxt(path, dtype = 'str')
    np.reshape(labels, (-1, 1))
    labels = np.asarray(labels)
    tmp = pd.get_dummies(labels)
    int_labels = tmp.values.argmax(1)
    return int_labels
    

In [ ]:
start_time = datetime.now()
print("Reading Images @ " + str(start_time))
imlist = get_resized_images(IMG_PATH) # imlist contains a list of image numpy arrays
end_time = datetime.now()
print("Complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

In [ ]:
print(imlist[0].shape)

In [ ]:
labels = get_labels(LABEL_PATH)
print(labels.shape)
print(labels[0:20])

In [ ]:
X = np.asarray(imlist)
y = labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', input_shape=[128, 128, 3]))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
opt = RMSprop(lr=0.0001, decay=1e-6)


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

model.fit_generator(datagen.flow(X_train, y_train,
                    batch_size=128),
                    epochs=50,
                    validation_data=(X_test, y_test),
                    workers=4)